# CAN-LSS-Mamba vast.ai Workflow

This notebook provides a complete workflow for training and evaluating the CAN-LSS-Mamba model on vast.ai.

## Prerequisites
- Repository cloned to `/workspace/can-lss-mamba`
- Dataset uploaded to `/workspace/data/can-train-and-test-v1.5/set_01/`

## Cell 1: Setup Environment

Run this cell once after cloning the repository.

In [ ]:
import os
os.chdir('/workspace/can-lss-mamba')

# Run setup script
!bash setup.sh

## Cell 2: Verify Environment

Check that all dependencies are installed and GPU is available.

In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn import __version__ as sklearn_version

print("=" * 60)
print("Environment Verification")
print("=" * 60)
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Scikit-learn Version: {sklearn_version}")

try:
    import mamba_ssm
    print("Mamba-SSM: Installed ✓")
except ImportError:
    print("Mamba-SSM: Not installed ✗")

try:
    import wandb
    print(f"WandB: {wandb.__version__}")
except ImportError:
    print("WandB: Not installed (optional)")

print("=" * 60)

## Cell 3: Configure Experiment

Set your experiment configuration and WandB credentials (optional).

In [ ]:
# Configuration
CONFIG_PATH = "configs/vastai.yaml"

# WandB Configuration (optional)
ENABLE_WANDB = False  # Set to True to enable WandB tracking
WANDB_API_KEY = "your_wandb_api_key_here"  # Get from https://wandb.ai/authorize

# Training Hyperparameters (optional overrides)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001

# Set environment variables
os.environ["CONFIG_PATH"] = CONFIG_PATH
os.environ["BATCH_SIZE"] = str(BATCH_SIZE)
os.environ["EPOCHS"] = str(EPOCHS)
os.environ["LR"] = str(LEARNING_RATE)

if ENABLE_WANDB:
    os.environ["WANDB_ENABLED"] = "true"
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    print("✅ WandB enabled")
else:
    os.environ["WANDB_ENABLED"] = "false"
    print("ℹ️  WandB disabled")

print(f"Configuration: {CONFIG_PATH}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")

## Cell 4: Preprocess Data

Preprocess the raw CAN bus data into training/validation windows.

In [ ]:
print("Starting preprocessing...")
!python preprocessing/CAN_preprocess.py
print("\n✅ Preprocessing complete!")

## Cell 5: Train Model

Train the LSS-CAN-Mamba model with WandB tracking (if enabled).

In [ ]:
print("Starting training...")
!python train.py
print("\n✅ Training complete!")

## Cell 6: Evaluate Model

Evaluate the trained model on all test scenarios.

In [ ]:
print("Starting evaluation...")
!python evaluate.py
print("\n✅ Evaluation complete!")

## Cell 7: View Results

Display evaluation results.

In [ ]:
import pandas as pd

results_file = "/workspace/final_thesis_results_02.csv"
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
    print("\n" + "="*60)
    print("Evaluation Results")
    print("="*60)
    print(df.to_string(index=False))
    print("\n" + "="*60)
    print(f"Average F1 Score: {df['F1_Score'].mean():.4f}")
    print(f"Average Accuracy: {df['Accuracy'].mean():.4f}")
    print("="*60)
else:
    print(f"Results file not found: {results_file}")

## Cell 8: Download Checkpoints (Optional)

Download model checkpoints to your local machine.

**Note:** In vast.ai Jupyter, you can download files by:
1. Navigate to `/workspace/checkpoints/set_01/` in the file browser
2. Right-click on `lss_can_mamba_best.pth` → Download

In [ ]:
# List checkpoint files
checkpoint_dir = "/workspace/checkpoints/set_01"
if os.path.exists(checkpoint_dir):
    print(f"Checkpoint files in {checkpoint_dir}:")
    for f in os.listdir(checkpoint_dir):
        file_path = os.path.join(checkpoint_dir, f)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"  - {f} ({size_mb:.2f} MB)")
else:
    print(f"Checkpoint directory not found: {checkpoint_dir}")

## Next Steps

1. **View Training Progress:** If WandB is enabled, visit: https://wandb.ai/YOUR_USERNAME/can-lss-mamba
2. **Download Checkpoints:** Use the file browser to download model files
3. **Re-run Training:** Modify hyperparameters in Cell 3 and re-run from Cell 5
4. **Try Different Configs:** Change `CONFIG_PATH` to experiment with different settings